In [596]:
import numpy as np
import pandas as pd
import os

In [597]:
Balance_Sheet = [
                "Total Share Capital",
                "Reserves and Surplus",
                "Total Reserves and Surplus",
                "Total Shareholders Funds",
                "Other Current Liabilities",
                "Total Current Liabilities",
                "Total Capital And Liabilities",
                "Tangible Assets",
                "Fixed Assets",
                "Total Non-Current Assets",
                "Total Current Assets",
                "Total Assets",
                ]

Cash_Flow = [
                "Net Profit/Loss Before Extraordinary Items And Tax",
                "Net CashFlow From Operating Activities",
                "Net Cash Used In Investing Activities",
                "Net Cash Used From Financing Activities",
                "Foreign Exchange Gains / Losses",
                "Net Inc/Dec In Cash And Cash Equivalents",
                "Cash And Cash Equivalents Begin of Year",
                "Cash And Cash Equivalents End Of Year",
                ]

Profit_Loss = [ 
                "Revenue From Operations [Gross]",
                "Revenue From Operations [Net]",
                "Total Operating Revenues",
                "Other Income",
                "Total Revenue",
                "Operating And Direct Expenses"
                "Employee Benefit Expenses",
                "Finance Costs",
                "Depreciation And Amortisation Expenses",
                "Other Expenses",
                "Total Expenses",
                "Profit/Loss Before Exceptional, ExtraOrdinary Items And Tax",
                "Profit/Loss Before Tax",
                "Current Tax",
                "Deferred Tax",
                "Total Tax Expenses",
                "Profit/Loss After Tax And Before ExtraOrdinary Items",
                "Profit/Loss From Continuing Operations",
                "Profit/Loss For The Period",
                "Basic EPS (Rs.)",
                "Diluted EPS (Rs.)",
               ]

Quarterly = [        
                "Net Sales/Income from operations",
                "Total Income From Operations",
                "Employees Cost",
                "Depreciat",
                "Other Expenses",
                "P/L Before Other Inc. , Int., Excpt. Items & Tax",
                "Other Income",
                "P/L Before Int., Excpt. Items & Tax",
                "Interest",
                "P/L Before Exceptional Items & Tax",
                "P/L Before Tax",
                "Tax",
                "P/L After Tax from Ordinary Activities",
                "Net Profit/(Loss) For the Period",
                "Equity Share Capital",
                "Basic EPS",
                "Diluted EPS",
                "Basic EPS.",
                "Diluted EPS.",
        ]

Ratio = [        
                "Revenue from Operations/Share (Rs.)",
                "PBDIT/Share (Rs.)",
                "PBIT/Share (Rs.)",
                "PBT/Share (Rs.)",
                "Net Profit/Share (Rs.)",
                "PBDIT Margin (%)",
                "PBIT Margin (%)",
                "PBT Margin (%)",
                "Net Profit Margin (%)",
                "Return on Networth / Equity (%)",
                "Return on Assets (%)",
                "Total Debt/Equity (X)"
                "Dividend Payout Ratio (NP) (%)",
                "Dividend Payout Ratio (CP) (%)",
                "Earnings Retention Ratio (%)",
                "Enterprise Value (Cr.)",
                "EV/EBITDA (X)",
        ]

Year = [        
                "Net Sales/Income from operations",
                "Total Income From Operations",
                "Employees Cost",
                "Depreciat",
                "Other Expenses",
                "P/L Before Other Inc. , Int., Excpt. Items & Tax",
                "Other Income",
                "P/L Before Int., Excpt. Items & Tax",
                "Interest",
                "P/L Before Exceptional Items & Tax",
                "P/L Before Tax",
                "Tax",
                "P/L After Tax from Ordinary Activities",
                "Net Profit/(Loss) For the Period",
                "Equity Share Capital",
                "Basic EPS",
                "Diluted EPS",
                "Basic EPS.",
                "Diluted EPS.",
        ]

In [598]:
input_file = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting\3i Infotech Ltd\Excel\Balance-sheet_combined.xlsx"
output_file = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Data\Data Preporcessing\3i infotech\cleanedDataBS.xlsx"

In [599]:
df = pd.read_excel(input_file)
df.head(3)

,Balance Sheet of 3i Infotech(in Rs. Cr.),Mar 09,Mar 08,Mar 07,Mar 06,Mar 05,Mar 14,Mar 13,Mar 12,Mar 11,...,Mar 19,Mar 18,Mar 17,Mar 16,Mar 15,Mar 24,Mar 23,Mar 22,Mar 21,Mar 20
0,NaN,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,...,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths
1,EQUITIES AND LIABILITIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SHAREHOLDER'S FUNDS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [600]:
df.iloc[:,0]

0                                             NaN
1                        EQUITIES AND LIABILITIES
2                             SHAREHOLDER'S FUNDS
3                            Equity Share Capital
4                             Total Share Capital
                         ...                     
60    Non-Current Investments Quoted Market Value
61    Non-Current Investments Unquoted Book Value
62                            CURRENT INVESTMENTS
63        Current Investments Quoted Market Value
64        Current Investments Unquoted Book Value
Name: Balance Sheet of 3i Infotech(in Rs. Cr.), Length: 65, dtype: object

In [601]:

# Extract df_parameters column
df_parameters = df.iloc[:, 0].astype(str)  # Convert to string to avoid NaN issues
df_parameters = list(df_parameters)

In [602]:
print(df_parameters)
print(len(df_parameters))

['nan', 'EQUITIES AND LIABILITIES', "SHAREHOLDER'S FUNDS", 'Equity Share Capital', 'Total Share Capital', 'Reserves and Surplus', 'Total Reserves and Surplus', 'Total Shareholders Funds', 'NON-CURRENT LIABILITIES', 'Long Term Borrowings', 'Deferred Tax Liabilities [Net]', 'Other Long Term Liabilities', 'Long Term Provisions', 'Total Non-Current Liabilities', 'CURRENT LIABILITIES', 'Short Term Borrowings', 'Trade Payables', 'Other Current Liabilities', 'Short Term Provisions', 'Total Current Liabilities', 'Total Capital And Liabilities', 'ASSETS', 'NON-CURRENT ASSETS', 'Tangible Assets', 'Intangible Assets', 'Capital Work-In-Progress', 'Other Assets', 'Fixed Assets', 'Non-Current Investments', 'Deferred Tax Assets [Net]', 'Long Term Loans And Advances', 'Other Non-Current Assets', 'Total Non-Current Assets', 'CURRENT ASSETS', 'Current Investments', 'Inventories', 'Trade Receivables', 'Cash And Cash Equivalents', 'Short Term Loans And Advances', 'OtherCurrentAssets', 'Total Current Asset

In [603]:
print(Balance_Sheet)
print(len(Balance_Sheet))

['Total Share Capital', 'Reserves and Surplus', 'Total Reserves and Surplus', 'Total Shareholders Funds', 'Other Current Liabilities', 'Total Current Liabilities', 'Total Capital And Liabilities', 'Tangible Assets', 'Fixed Assets', 'Total Non-Current Assets', 'Total Current Assets', 'Total Assets']
12


In [604]:
not_common_parameters = []

for item in df_parameters:
    if item not in Balance_Sheet:
        not_common_parameters.append(item)
        
print(not_common_parameters)
print(len(not_common_parameters))

['nan', 'EQUITIES AND LIABILITIES', "SHAREHOLDER'S FUNDS", 'Equity Share Capital', 'NON-CURRENT LIABILITIES', 'Long Term Borrowings', 'Deferred Tax Liabilities [Net]', 'Other Long Term Liabilities', 'Long Term Provisions', 'Total Non-Current Liabilities', 'CURRENT LIABILITIES', 'Short Term Borrowings', 'Trade Payables', 'Short Term Provisions', 'ASSETS', 'NON-CURRENT ASSETS', 'Intangible Assets', 'Capital Work-In-Progress', 'Other Assets', 'Non-Current Investments', 'Deferred Tax Assets [Net]', 'Long Term Loans And Advances', 'Other Non-Current Assets', 'CURRENT ASSETS', 'Current Investments', 'Inventories', 'Trade Receivables', 'Cash And Cash Equivalents', 'Short Term Loans And Advances', 'OtherCurrentAssets', 'OTHER ADDITIONAL INFORMATION', 'CONTINGENT LIABILITIES, COMMITMENTS', 'Contingent Liabilities', 'CIF VALUE OF IMPORTS', 'Raw Materials', 'Stores, Spares And Loose Tools', 'Trade/Other Goods', 'Capital Goods', 'EXPENDITURE IN FOREIGN EXCHANGE', 'Expenditure In Foreign Currency',

In [605]:
index_to_drop =df[df.iloc[:, 0].isin(not_common_parameters)].index

In [606]:
df = df.drop(index_to_drop,axis=0)

In [607]:
df.head(5)

,Balance Sheet of 3i Infotech(in Rs. Cr.),Mar 09,Mar 08,Mar 07,Mar 06,Mar 05,Mar 14,Mar 13,Mar 12,Mar 11,...,Mar 19,Mar 18,Mar 17,Mar 16,Mar 15,Mar 24,Mar 23,Mar 22,Mar 21,Mar 20
0,NaN,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,...,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths,12 mths
4,Total Share Capital,230.75,230.54,156.30,153.05,181.00,637.64,636.55,291.99,291.99,...,"1,992.07","1,957.25","1,495.35",924.48,668.75,169.23,168.47,167.94,"2,047.16","2,047.16"
5,Reserves and Surplus,515.90,414.45,305.35,219.17,29.91,-40.34,397.11,372.29,776.25,...,"-1,259.07","-1,512.72",-963.34,-477.44,"-1,079.11",-81.97,735.67,681.79,-803.58,"-1,128.16"
6,Total Reserves and Surplus,515.90,414.45,305.35,219.17,29.91,-40.34,397.11,372.29,776.25,...,"-1,259.07","-1,512.72",-963.34,-477.44,-953.61,-81.97,735.67,681.79,-803.58,"-1,128.16"
7,Total Shareholders Funds,746.65,644.98,461.65,372.22,210.92,597.30,"1,033.66",664.28,"1,068.24",...,733.00,444.53,532.01,730.72,-284.86,87.26,904.14,849.73,"1,243.58",919.00


In [608]:
df.replace("12 mths",np.nan,inplace=True)
df.head(3)

,Balance Sheet of 3i Infotech(in Rs. Cr.),Mar 09,Mar 08,Mar 07,Mar 06,Mar 05,Mar 14,Mar 13,Mar 12,Mar 11,...,Mar 19,Mar 18,Mar 17,Mar 16,Mar 15,Mar 24,Mar 23,Mar 22,Mar 21,Mar 20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Total Share Capital,230.75,230.54,156.30,153.05,181.00,637.64,636.55,291.99,291.99,...,"1,992.07","1,957.25","1,495.35",924.48,668.75,169.23,168.47,167.94,"2,047.16","2,047.16"
5,Reserves and Surplus,515.90,414.45,305.35,219.17,29.91,-40.34,397.11,372.29,776.25,...,"-1,259.07","-1,512.72",-963.34,-477.44,"-1,079.11",-81.97,735.67,681.79,-803.58,"-1,128.16"


In [609]:
df.dropna(inplace=True)
# df.reset_index(drop=True, inplace=True)

In [610]:
df.iloc[:, 1:] = df.iloc[:, 1:].replace(",", "", regex=True)

In [611]:
df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

In [612]:
column_start = df.columns[0]
column_start

'Balance Sheet of 3i Infotech(in Rs. Cr.)'

In [613]:
df.columns[:]

Index(['Balance Sheet of 3i Infotech(in Rs. Cr.)', 'Mar 09', 'Mar 08',
       'Mar 07', 'Mar 06', 'Mar 05', 'Mar 14', 'Mar 13', 'Mar 12', 'Mar 11',
       'Mar 10', 'Mar 19', 'Mar 18', 'Mar 17', 'Mar 16', 'Mar 15', 'Mar 24',
       'Mar 23', 'Mar 22', 'Mar 21', 'Mar 20'],
      dtype='object')

In [614]:
# Extract the numeric part (years) and sort
sorted_columns = sorted(df.columns[1:], key=lambda x: int(x.split()[1]))

# Reorder the DataFrame columns
df = df[[column_start] + sorted_columns]

# Display the first few rows
df.head(3)


,Balance Sheet of 3i Infotech(in Rs. Cr.),Mar 05,Mar 06,Mar 07,Mar 08,Mar 09,Mar 10,Mar 11,Mar 12,Mar 13,...,Mar 15,Mar 16,Mar 17,Mar 18,Mar 19,Mar 20,Mar 21,Mar 22,Mar 23,Mar 24
4,Total Share Capital,181.0,153.05,156.3,230.54,230.75,268.76,291.99,291.99,636.55,...,668.75,924.48,1495.35,1957.25,1992.07,2047.16,2047.16,167.94,168.47,169.23
5,Reserves and Surplus,29.91,219.17,305.35,414.45,515.9,626.34,776.25,372.29,397.11,...,-1079.11,-477.44,-963.34,-1512.72,-1259.07,-1128.16,-803.58,681.79,735.67,-81.97
6,Total Reserves and Surplus,29.91,219.17,305.35,414.45,515.9,626.34,776.25,372.29,397.11,...,-953.61,-477.44,-963.34,-1512.72,-1259.07,-1128.16,-803.58,681.79,735.67,-81.97


In [615]:
df = df.T.reset_index()
df

,index,4,5,6,7,17,19,20,23,27,32,40,41
0,Balance Sheet of 3i Infotech(in Rs. Cr.),Total Share Capital,Reserves and Surplus,Total Reserves and Surplus,Total Shareholders Funds,Other Current Liabilities,Total Current Liabilities,Total Capital And Liabilities,Tangible Assets,Fixed Assets,Total Non-Current Assets,Total Current Assets,Total Assets
1,Mar 05,181.0,29.91,29.91,210.92,14.0,86.6,447.06,13.35,173.06,279.45,167.6,447.06
2,Mar 06,153.05,219.17,219.17,372.22,16.09,111.78,801.31,-9.63,209.38,332.04,469.27,801.31
3,Mar 07,156.3,305.35,305.35,461.65,12.97,133.11,1131.73,41.55,137.79,756.35,375.38,1131.73
4,Mar 08,230.54,414.45,414.45,644.98,20.0,170.76,2069.52,45.2,249.38,1492.13,577.4,2069.52
5,Mar 09,230.75,515.9,515.9,746.65,145.62,467.05,2657.52,219.62,452.16,1967.23,690.29,2657.52
6,Mar 10,268.76,626.34,626.34,895.1,184.0,520.38,2845.11,22.64,231.23,2048.42,796.69,2845.11
7,Mar 11,291.99,776.25,776.25,1068.24,318.03,1007.63,3270.15,120.57,217.83,2408.2,861.95,3270.15
8,Mar 12,291.99,372.29,372.29,664.28,172.26,672.71,3578.55,307.77,798.63,3009.22,569.33,3578.55
9,Mar 13,636.55,397.11,397.11,1033.66,804.76,978.37,4003.04,275.3,1534.68,3752.0,251.04,4003.04


In [616]:
new_df = df[1:].reset_index()
new_df

,level_0,index,4,5,6,7,17,19,20,23,27,32,40,41
0,1,Mar 05,181.0,29.91,29.91,210.92,14.0,86.6,447.06,13.35,173.06,279.45,167.6,447.06
1,2,Mar 06,153.05,219.17,219.17,372.22,16.09,111.78,801.31,-9.63,209.38,332.04,469.27,801.31
2,3,Mar 07,156.3,305.35,305.35,461.65,12.97,133.11,1131.73,41.55,137.79,756.35,375.38,1131.73
3,4,Mar 08,230.54,414.45,414.45,644.98,20.0,170.76,2069.52,45.2,249.38,1492.13,577.4,2069.52
4,5,Mar 09,230.75,515.9,515.9,746.65,145.62,467.05,2657.52,219.62,452.16,1967.23,690.29,2657.52
5,6,Mar 10,268.76,626.34,626.34,895.1,184.0,520.38,2845.11,22.64,231.23,2048.42,796.69,2845.11
6,7,Mar 11,291.99,776.25,776.25,1068.24,318.03,1007.63,3270.15,120.57,217.83,2408.2,861.95,3270.15
7,8,Mar 12,291.99,372.29,372.29,664.28,172.26,672.71,3578.55,307.77,798.63,3009.22,569.33,3578.55
8,9,Mar 13,636.55,397.11,397.11,1033.66,804.76,978.37,4003.04,275.3,1534.68,3752.0,251.04,4003.04
9,10,Mar 14,637.64,-40.34,-40.34,597.3,482.14,826.45,4488.99,217.69,1426.48,3791.29,697.7,4488.99


In [617]:
# df.columns = df.iloc[0] 
# df.columns

In [618]:
# # Transpose the DataFrame
# df = df.T  

# # Set the first row as column headers
# df.columns = df.iloc[0]  

# # Remove the first row and reset index
# df = df[1:].reset_index()

# # Rename the new first column to "Date" or as needed
# df.rename(columns={'index': 'Date'}, inplace=True)

# # Save the transformed DataFrame
# df.to_excel("transformed_file.xlsx", index=False)

# df.head()

In [ ]:
output_file = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Data\Data Preporcessing\3i infotech\cleanedDataBS.xlsx"
df.to_excel(output_file,index=False)

In [620]:
df.to_excel(output_file,index=False)
print("File saved successfully in",output_file)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\sharm\\OneDrive\\Desktop\\Kishan\\Data\\Data Preporcessing\\3i infotech\\cleanedDataBS.xlsx'